In [1]:
import os
import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib, os, random
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions


In [2]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [3]:
os.listdir('FishImgDataset')

['test', 'train', 'val']

In [4]:
train = 'FishImgDataset/train'
validation = 'FishImgDataset/val'
test = 'FishImgDataset/test' 

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2,  
                                   zoom_range = 0.2,  
                                   horizontal_flip = True)

In [6]:
valid_datagen = ImageDataGenerator(rescale = 1.0/255.) 
test_datagen = ImageDataGenerator(rescale = 1.0/255.) 

In [7]:
train_generator = train_datagen.flow_from_directory(train, target_size=(224,224),batch_size=32,class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(validation, target_size=(224,224),batch_size=32,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test, target_size=(224,224),batch_size=32,class_mode='categorical',shuffle=False)

Found 8791 images belonging to 31 classes.
Found 2751 images belonging to 31 classes.
Found 1760 images belonging to 31 classes.


In [8]:
inception = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))

87910968/87910968 [==============================] - 29s 0us/step


In [9]:
inception.trainable = True
for layer in inception.layers[:197]:
    layer.trainable = False 

In [10]:
for idx, layer in enumerate(inception.layers):
    print(f' {idx}:  {layer.name}: trainable = {layer.trainable}')

 0:  input_1: trainable = False
 1:  conv2d: trainable = False
 2:  batch_normalization: trainable = False
 3:  activation: trainable = False
 4:  conv2d_1: trainable = False
 5:  batch_normalization_1: trainable = False
 6:  activation_1: trainable = False
 7:  conv2d_2: trainable = False
 8:  batch_normalization_2: trainable = False
 9:  activation_2: trainable = False
 10:  max_pooling2d: trainable = False
 11:  conv2d_3: trainable = False
 12:  batch_normalization_3: trainable = False
 13:  activation_3: trainable = False
 14:  conv2d_4: trainable = False
 15:  batch_normalization_4: trainable = False
 16:  activation_4: trainable = False
 17:  max_pooling2d_1: trainable = False
 18:  conv2d_8: trainable = False
 19:  batch_normalization_8: trainable = False
 20:  activation_8: trainable = False
 21:  conv2d_6: trainable = False
 22:  conv2d_9: trainable = False
 23:  batch_normalization_6: trainable = False
 24:  batch_normalization_9: trainable = False
 25:  activation_6: trainab

In [11]:
last_layer = inception.get_layer('mixed7')

In [12]:
layer_output = last_layer.output

In [13]:
n_categories = len(os.listdir('FishImgDataset/train'))

In [14]:
xs = Flatten()(layer_output)
xs = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(xs)
xs = Dropout(0.4)(xs)
xs = Dense(n_categories, activation='softmax')(xs)

model = Model(inputs=inception.inputs, outputs=xs)

In [15]:
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [16]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [17]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 10, #10
            callbacks=[callback])

Epoch 1/10
275/275 [==============================] - 1341s 5s/step - loss: 2.1639 - accuracy: 0.6756 - val_loss: 1.0030 - val_accuracy: 0.9200 - lr: 1.0000e-04
Epoch 2/10
275/275 [==============================] - 1277s 5s/step - loss: 1.0066 - accuracy: 0.8914 - val_loss: 0.6806 - val_accuracy: 0.9673 - lr: 1.0000e-04
Epoch 3/10
275/275 [==============================] - 1282s 5s/step - loss: 0.7053 - accuracy: 0.9346 - val_loss: 0.5130 - val_accuracy: 0.9786 - lr: 1.0000e-04
Epoch 4/10
275/275 [==============================] - 1284s 5s/step - loss: 0.5351 - accuracy: 0.9501 - val_loss: 0.4101 - val_accuracy: 0.9807 - lr: 1.0000e-04
Epoch 5/10
275/275 [==============================] - 1275s 5s/step - loss: 0.4195 - accuracy: 0.9619 - val_loss: 0.3155 - val_accuracy: 0.9851 - lr: 1.0000e-04
Epoch 6/10
275/275 [==============================] - 1268s 5s/step - loss: 0.3362 - accuracy: 0.9669 - val_loss: 0.2751 - val_accuracy: 0.9851 - lr: 1.0000e-04
Epoch 7/10
275/275 [==============

In [18]:

model_name = 'FishModelClassifier_V6.h5'
model.save(model_name, save_format='h5')
model.save_weights('model_weights_V6.h5')


In [19]:
class_name=['Bangus', 'Big Head Carp', 'Black Spotted Barb', 'Catfish', 'Climbing Perch', 'Fourfinger Threadfin',
        'Freshwater Eel', 'Glass Perchlet', 'Goby', 'Gold Fish', 'Gourami', 'Grass Carp', 'Green Spotted Puffer',
        'Indian Carp', 'Indo-Pacific Tarpon', 'Jaguar Gapote', 'Janitor Fish', 'Knifefish', 'Long-Snouted Pipefish',
        'Mosquito Fish', 'Mudfish', 'Mullet', 'Pangasius', 'Perch', 'Scat Fish', 'Silver Barb', 'Silver Carp',
        'Silver Perch', 'Snakehead', 'Tenpounder', 'Tilapia']


In [25]:
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import numpy as np

In [26]:
def predict(path):
    img=load_img(path,target_size=(224,224,3))#convert image size and declare the kernel
    img=img_to_array(img)#convert the image to an image array
    img=img/255 #rgb is 255
    img=np.expand_dims(img,[0])
    answer=model.predict(img)
    x = list(np.argsort(answer[0])[::-1][:5])
    
    for i in x:
        print("{className}: {predVal:.2f}%".format(className=class_name[i], predVal=float(answer[0][i])*100))
    
    y_class=answer.argmax(axis=-1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = class_name[y]
    
    return res

In [30]:
img='C:\\Users\\hp\\FishImgDataset\\train\\Catfish\\0b17caaa-b3a0-4302-a59b-4b02da660189-1200mm.jpg'
img2='C:\\Users\\hp\Downloads\\catfish.jpeg'
print(predict(img))

1/1 [==============================] - 0s 284ms/step
Catfish: 100.00%
Grass Carp: 0.00%
Freshwater Eel: 0.00%
Big Head Carp: 0.00%
Knifefish: 0.00%
Catfish
